<a href="https://colab.research.google.com/github/mantoan-thi/-Aprendizagem-por-Refor-o-Profunda-para-Negocia-o-no-Mercado-de-A-es/blob/main/Aprendizagem_por_Refor%C3%A7o_Profunda_para_Negocia%C3%A7%C3%A3o_no_Mercado_de_A%C3%A7%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Etapa 1: Instalações da bibliotecas

In [ ]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [ ]:
!pip install pandas-datareader

In [ ]:
!pip uninstall -y tensorflow
#!pip install tensorflow

In [33]:
!pip install --upgrade pandas
!pip install --upgrade pandas-datareader

     |████████████████████████████████| 11.3 MB 5.2 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.2 which is incompatible.


     |████████████████████████████████| 109 kB 5.4 MB/s 
  Attempting uninstall: pandas-datareader
    Found existing installation: pandas-datareader 0.9.0
    Uninstalling pandas-datareader-0.9.0:
      Successfully uninstalled pandas-datareader-0.9.0


# Etapa 2: importação das bibliotecas

In [1]:
import math                              # Para construção da função sigmoid
import random                            # Para números aleatórios  
import numpy as np
import pandas as pd
import tensorflow as tf                  
import matplotlib.pyplot as plt
import pandas_datareader as data_reader   
from tqdm import tqdm_notebook,tqdm       # Para barra de progresso mais intuitivas
from collections import deque             # Para memória de re-play

In [2]:
tf.__version__

AttributeError: ignored

# Etapa 3: Construção da IA para anegocição das ações

In [3]:
class AI_Trader():
  def __init__(self,state_size,action_space = 3,model_name = "AITrader"):
    self.state_size = state_size       # São as informações que estão vindo do ambiente, camada de entrada da rede neural
    self.action_space = action_space   # São as ações: comprar, vender ou não fazer nada. 
    self.memory = deque(maxlen=2000)   # Experiência de re-play
    self.model_name = model_name

    self.gamma = 0.95                  #
    self.epsilon = 1.0                 # 100% indicar que todas as ações são randomicas, sem utilizar a rede neural
    self.epsilon_final = 0.01          # 
    self.epsilon_decay = 0.995         # Decrementar os valor de espilon

    def model_builder(self):
      model = tf.keras.models.sequential()
      model.add(tf.keras.layers.Dense(units=32,activation = "relu",input_dim = self.state_size))
      model.add(tf.keras.layers.Dense(units=64,activation = "relu"))
      model.add(tf.keras.layers.Dense(units=128,activation = "relu"))
      model.add(tf.keras.layers.Dense(units = self.action_space,activation = 'linear'))
      model.compile(loss='mse',optimizer = tf.keras.optimizer.Adam(lr=0.001))
      return model

    def trade(self,state):
      if random.random() <= self.epsilon:
        return random.randrange(self.action_space)
      actions = self.model.predict(state)
      return np.argmax(actions[0])
  
    def batch_train(self,batch_size):
      batch = []
      for i in range(en(self.memory)-batch_size+1,len(self.memory)):
        batch.append(self.memory)
      for state, action, reward, next_state, done in batch:
        if not done:
          reward = reward +self.gamma * np.amax(self.model.predict(next_state)[0])
        
        target = self.model.predict(state)
        target[0][action] = reward

        self.model.fit(state,target, epochs=1, verbose=0)
      if self.epsilon > self.epsilon_final:
        self.epsilon*=self.epsilon_decay


# Etapa 4: Pré-processamento da base de dados

# Definição de funções auxiliares

Sigmoid

In [4]:
def sigmoid(x):
  return 1 /(1 + math.exp(-x))

Formatação de preços

In [6]:
def stocks_prices_format(n): # verifica se vc esta tendo ou não lucro
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

Carregador de base de dados

In [22]:
# Empresas: SABESP,    ELETROBRÁS,  ENERGISA, B3
tickers = {'SBSP3.SA', 'ENGI3.SA', 'ELET3.SA','^BVSP'}
stock_name = list(tickers)[0]

In [27]:
def dataset_loader(stock_name):
  dataset = data_reader.DataReader(stock_name,data_source="yahoo")
  star_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  close = dataset['Close']
  return close

Criador de estados

In [33]:
def state_creator(data,timestep,window_size):
  starting_id = timestep - window_size+1
  if starting_id >=0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = -starting_id * [data[0]+list(data[0:timestep+1])]
  state = []
  for i in range(window_size-1):
    state.append(sigmoid(windowed_data[i+1]-windowed_data[i]))
  return np.array([state])

Carregando a base de dados